# Can we predict business registration?

In [152]:
import pandas as pd 
import numpy as np

# Tratamentos iniciais

1) Lendo os dados 

In [153]:
df = pd.read_excel("Base_completa_tratada.xlsx")

2) Selecionando as variávies que vamos utilizar

In [154]:
df = df[['qtd_abertas_Empresario_Individual','Mecanismos_incentivo','Reducao_IPTU', 'Isencao_IPTU', 'Reducao_ISSQN','Isencao_ISSQN','Isencao_Tx','Cessao_terrenos',
    'Doacao_terrenos','Outros_mecanismos','taxa_homicidio','RH_adm_dir','ISH','densidade_banda_larga','densidade_telefonia_movel','qtd_cursos_engenharias',
    'qtd_cursos_negocios_direito','media_notas_CN','media_notas_CH','media_NU_NOTA_LC','media_NU_NOTA_MT',
    'media_NU_NOTA_REDACAO','valor_medio_recebido_familia','pib_per_capita_preco_corrente','perc_20_59_anos','pop_total','total_sim_tecnologia']]

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   qtd_abertas_Empresario_Individual  5570 non-null   int64  
 1   Mecanismos_incentivo               5568 non-null   object 
 2   Reducao_IPTU                       3484 non-null   object 
 3   Isencao_IPTU                       3484 non-null   object 
 4   Reducao_ISSQN                      3484 non-null   object 
 5   Isencao_ISSQN                      3484 non-null   object 
 6   Isencao_Tx                         3484 non-null   object 
 7   Cessao_terrenos                    3484 non-null   object 
 8   Doacao_terrenos                    3484 non-null   object 
 9   Outros_mecanismos                  3484 non-null   object 
 10  taxa_homicidio                     3890 non-null   float64
 11  RH_adm_dir                         5547 non-null   float

3) Tratamentos iniciais 

Transformando o nome do município no índice da tabela

In [156]:
#df = df.set_index('NMMUN')
df.head()

,qtd_abertas_Empresario_Individual,Mecanismos_incentivo,Reducao_IPTU,Isencao_IPTU,Reducao_ISSQN,Isencao_ISSQN,Isencao_Tx,Cessao_terrenos,Doacao_terrenos,Outros_mecanismos,...,media_notas_CN,media_notas_CH,media_NU_NOTA_LC,media_NU_NOTA_MT,media_NU_NOTA_REDACAO,valor_medio_recebido_familia,pib_per_capita_preco_corrente,perc_20_59_anos,pop_total,total_sim_tecnologia
0,194,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,497.721951,504.034091,491.079545,543.558537,621.363636,18495,25091.18,0.587227,22516,18.0
1,1275,Sim,Sim,Sim,Não,Não,Sim,Não,Sim,Sim,...,474.773986,495.727950,484.710248,518.811149,584.534161,17615,25730.20,0.591797,111148,12.0
2,49,Sim,Não,Não,Não,Sim,Não,Não,Não,Sim,...,477.266667,486.071429,447.628571,522.883333,471.428571,18956,32226.25,0.584567,5067,8.0
3,1258,Sim,Não,Sim,Sim,Não,Sim,Sim,Sim,Não,...,489.744000,505.006061,490.991919,533.458933,611.717172,19376,29331.29,0.595584,86416,17.0
4,257,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,...,479.268750,477.045714,454.722857,480.696875,576.571429,17752,37069.25,0.598272,16088,8.0


Vamos fazer alguns tratamentos para algumas variáveis. 
- Nas variáveis sobre taxa de homicídios, cursos de engenharia ou negócios/direito, vamos transformar os "NA" em 0
- Nas variáveis de políticas municipais, adicionamos "Não" aos "NA"
- Nas variáveis do Enem, substituímos o valor pela mediana

In [157]:
df['qtd_cursos_engenharias'] = df['qtd_cursos_engenharias'].fillna(0)

df['qtd_cursos_negocios_direito'] = df['qtd_cursos_negocios_direito'].fillna(0)

df['total_sim_tecnologia'] = df['total_sim_tecnologia'].fillna(0)

df['taxa_homicidio'] = df['taxa_homicidio'].fillna(0)

In [158]:
df[['Mecanismos_incentivo','Reducao_IPTU', 'Isencao_IPTU', 'Reducao_ISSQN','Isencao_ISSQN','Isencao_Tx','Cessao_terrenos',
    'Doacao_terrenos','Outros_mecanismos']] = df[['Mecanismos_incentivo','Reducao_IPTU', 'Isencao_IPTU', 'Reducao_ISSQN','Isencao_ISSQN','Isencao_Tx','Cessao_terrenos',
    'Doacao_terrenos','Outros_mecanismos']].fillna('Não')

In [159]:
df[['media_notas_CN','media_notas_CH','media_NU_NOTA_LC','media_NU_NOTA_MT',
    'media_NU_NOTA_REDACAO']] = df[['media_notas_CN','media_notas_CH','media_NU_NOTA_LC','media_NU_NOTA_MT',
    'media_NU_NOTA_REDACAO']].fillna(df[['media_notas_CN','media_notas_CH','media_NU_NOTA_LC','media_NU_NOTA_MT',
    'media_NU_NOTA_REDACAO']].median())


In [160]:
df = pd.get_dummies(df, drop_first=True)

In [172]:
df = df[np.isfinite(df).all(1)]

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   qtd_abertas_Empresario_Individual  5570 non-null   int64  
 1   taxa_homicidio                     5570 non-null   float64
 2   RH_adm_dir                         5547 non-null   float64
 3   densidade_banda_larga              5570 non-null   float64
 4   densidade_telefonia_movel          5570 non-null   float64
 5   qtd_cursos_engenharias             5570 non-null   float64
 6   qtd_cursos_negocios_direito        5570 non-null   float64
 7   media_notas_CN                     5570 non-null   float64
 8   media_notas_CH                     5570 non-null   float64
 9   media_NU_NOTA_LC                   5570 non-null   float64
 10  media_NU_NOTA_MT                   5570 non-null   float64
 11  media_NU_NOTA_REDACAO              5570 non-null   float

4) Separando variável dependente e variáveis independentes 

In [173]:
X = df.drop('qtd_abertas_Empresario_Individual', axis = 1)
y = df['qtd_abertas_Empresario_Individual']

In [163]:
# Não entendi pq vc está fazendo esse procedimento. Não tem necessidade. O train_test_split já serve para isso 
#df_div = np.random.rand(len(df_get)) < 0.8 # cria uma mascara com as posicoes
#train = df_get[df_div]
#test = df_get[~df_div]

In [164]:
#X = train
#y = train["qtd_abertas"] 
#X_test = test

In [165]:
#X = X.drop("qtd_abertas", axis = 1)

# Rodando algoritmos

In [195]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

In [174]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35,
                                                    random_state=42)

# vamos criar uma variável para região do município para usar no stratify. Eu tentei usar a variável SGUF, mas como 
# DF só tem um caso, ele não deixou eu aplicar

In [175]:
list(X_train.select_dtypes(include = ['float64','int64']))

['taxa_homicidio',
 'RH_adm_dir',
 'densidade_banda_larga',
 'densidade_telefonia_movel',
 'qtd_cursos_engenharias',
 'qtd_cursos_negocios_direito',
 'media_notas_CN',
 'media_notas_CH',
 'media_NU_NOTA_LC',
 'media_NU_NOTA_MT',
 'media_NU_NOTA_REDACAO',
 'valor_medio_recebido_familia',
 'pib_per_capita_preco_corrente',
 'perc_20_59_anos',
 'pop_total',
 'total_sim_tecnologia']

In [176]:
# https://stackoverflow.com/questions/48673402/how-can-i-standardize-only-numeric-variables-in-an-sklearn-pipeline
# criei esse procedimento para padronizar o valor das variáveis 

scaler = StandardScaler()

numeric_features = list(X_train.select_dtypes(include = ['float64','int64']))

X_train.loc[:,numeric_features] = scaler.fit_transform(X_train.loc[:,numeric_features])

X_train.head()

,taxa_homicidio,RH_adm_dir,densidade_banda_larga,densidade_telefonia_movel,qtd_cursos_engenharias,qtd_cursos_negocios_direito,media_notas_CN,media_notas_CH,media_NU_NOTA_LC,media_NU_NOTA_MT,...,Reducao_ISSQN_Sim,Isencao_ISSQN_Sim,Isencao_Tx_Sim,Cessao_terrenos_Sim,Doacao_terrenos_Sim,Outros_mecanismos_Sim,ISH_Baixa,ISH_Máxima,ISH_Média,ISH_Mínima
869,-0.243556,-0.309291,-0.886375,-0.322067,-0.089636,-0.079327,-1.517821,-2.027806,-2.072030,-1.815449,...,0,0,0,0,0,0,0,0,1,0
1223,-0.243556,-0.282632,-1.078951,-0.667171,-0.089636,-0.079327,0.479304,0.331024,0.844911,-0.345687,...,0,0,0,0,0,0,0,0,0,0
1414,-0.211621,-0.237213,-1.033721,-1.182348,-0.089636,-0.079327,-1.210267,-1.497413,-1.778589,-0.792296,...,0,0,1,0,0,0,0,0,1,0
3631,-0.243556,-0.261239,0.638662,0.966549,-0.089636,-0.079327,1.591120,1.265923,1.064882,-0.130288,...,0,0,0,0,0,0,0,1,0,0
410,-0.179686,-0.203972,-0.734349,-1.575174,-0.089636,-0.079327,-1.405162,-1.098686,-1.234158,-1.035150,...,0,0,0,0,0,0,0,0,0,0


### 1) Regressão linear

In [186]:
reg = LinearRegression().fit(X_train, y_train)

In [187]:
y_pred = reg.predict(X_test)

In [188]:
mae_rl = mean_absolute_error(y_true=y_test,y_pred=y_pred)

In [189]:
mape_rl = mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)

In [200]:
rmse_rl = mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)

In [201]:
print("MAE:",mae_rl)
print("MAPE:",mape_rl)
print("RMSE:",rmse_rl)

MAE: 105078850.38454518
MAPE: 469145.5971097201
RMSE: 446048550.2311395


Floresta Aleatória - Classificação

In [72]:
#bibliotecas Floresta aleatória 
from  sklearn.ensemble  import RandomForestClassifier 

from sklearn.model_selection import train_test_split


In [73]:
#Separando em treino e teste

X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size = 0.3, random_state= 0) # 30%

In [ ]:
#floresta aleatória - Classificação 

modelo_floresta_aleatoria = RandomForestClassifier(n_estimators=1000,random_state = 0) 

modelo_floresta_aleatoria.fit (X_treino, y_treino)


#RandomForestClassifier não aceita valores ausentes identificados como NA

In [ ]:
#previsão do modelo
#previsao = modelo_floresta_aleatoria.predict(X_valid)

In [ ]:
#Acuracia

#Acuracia = np.mean(y_valid == previsao) 

#print(Acuracia)

Arvore de Decisão - Regressão

In [81]:
#biblioteca - Arvore de Decisão (regressão)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [82]:
#separando em treino e teste

X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size = 0.3, random_state= 0) # 30%

In [ ]:
#max_depth = quantidade de divisões

arvore_decisao = DecisionTreeRegressor( max_depth= 3, random_state= 0) #OUTRO PARAMENTO -> min_samples_leaf
arvore_decisao.fit(X_treino, y_treino)

In [ ]:
#pred_arv_decisao = arvore_decisao.predict(X_valid)

#np.sqrt (mean_squared_error (y_valid, pred_arv_decisao))

Testando o  HistGradientBoostingClassifier


In [84]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [86]:
gradiente = HistGradientBoostingClassifier (max_depth= 3, random_state= 0)
gradiente.fit(X_treino, y_treino)

HistGradientBoostingClassifier(max_depth=3, random_state=0)

In [89]:
#previsão do modelo
previsao = gradiente.predict(X_valid)

In [91]:
#Acuracia

Acuracia = np.mean(y_valid == previsao) 

print(Acuracia)

0.008233532934131737


Testando a base com as variaveis categoricas 


In [98]:
df_category = df[['qtd_abertas','SGUF','NMMUN','Mecanismos_incentivo','Reducao_IPTU', 'Isencao_IPTU', 'Reducao_ISSQN','Isencao_ISSQN','Isencao_Tx',
         'Cessao_terrenos','Doacao_terrenos','Outros_mecanismos','ISH','atividade_maior_valor_ad_bruto']]

In [99]:
df_get_2 = pd.get_dummies(df_category , prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

In [100]:
df_div = np.random.rand(len(df_get_2)) < 0.8 # cria uma mascara com as posicoes
train = df_get[df_div]
test = df_get[~df_div]

In [101]:
X = train
y = train["qtd_abertas"] 
X_test = test

In [102]:
X = X.drop("qtd_abertas", axis = 1)

HistGradientBoostingClassifier

In [104]:
#separando em treino e teste

X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size = 0.3, random_state= 0) # 30%

In [103]:
gradiente = HistGradientBoostingClassifier (max_depth= 3, random_state= 0)
gradiente.fit(X_treino, y_treino)

HistGradientBoostingClassifier(max_depth=3, random_state=0)

In [105]:
#previsão do modelo
previsao = gradiente.predict(X_valid)

In [106]:
#Acuracia

Acuracia = np.mean(y_valid == previsao) 

print(Acuracia)

0.016454749439042633
